# NLP Course Assignment 3: Russian Nested Named Entities

This notebook was executed in [Google Colab](https://colab.research.google.com/)

For the task, I will use [spaCy](https://spacy.io/)

In [ ]:
import spacy

In [ ]:
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 863.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load("ru_core_news_lg")

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 25.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


Downloading the [dataset](https://huggingface.co/datasets/iluvvatar/RuNNE)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("iluvvatar/RuNNE")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for iluvvatar/RuNNE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/iluvvatar/RuNNE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 461
    })
    test: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 93
    })
    dev: Dataset({
        features: ['id', 'text', 'entities'],
        num_rows: 323
    })
})

Converting the data into Docs for spaCy

In [ ]:
from spacy.tokens import DocBin, Doc
from tqdm import tqdm

def make_doc(data):
    """
    Make the DocBin object based on data
    """
    
    doc_bin = DocBin()

    bar = tqdm(data)
    for row in bar:
        doc = nlp(row["text"])
        ents = [
            [int(x) if i < 2 else x for i, x in enumerate(line.split())]
              for line in row["entities"]
        ]
        
        # Filter out overlapping entities, because spaCy cannot work with them
        ents.sort(key=lambda x: (x[0], -x[1]))
        filtered_ents = []
        for ent in ents:
            if len(filtered_ents) == 0 or ent[0] > filtered_ents[-1][1]:
                filtered_ents.append(ent)
        doc.ents = filter(None, map(lambda x: doc.char_span(x[0], x[1], label=x[2]), filtered_ents))
        
        doc_bin.add(doc)

    return doc_bin

In [ ]:
make_doc(dataset["train"]).to_disk("train.spacy")
make_doc(dataset["dev"]).to_disk("dev.spacy")

100%|██████████| 323/323 [00:36<00:00,  8.82it/s]


Install [spacy-transformers](https://explosion.ai/blog/spacy-transformers) to be able to utilize transformers

In [ ]:
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 35.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

Make a config for training

In [ ]:
!python -m spacy init config config.cfg --pipeline ner --lang ru --gpu --force

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
ℹ Generated config template specific for your use case
- Language: ru
- Pipeline: ner
- Optimize for: efficiency
- Hardware: GPU
- Transformer: bert-base-multilingual-uncased
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Train the model!

In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output ./spacy_ner_model --gpu-id 0

ℹ Saving to output directory: spacy_ner_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        1110.46   1320.75    0.00    0.00    0.00    0.00


In [ ]:
import spacy_transformers

nlp_ner = spacy.load("spacy_ner_model/model-best")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
doc = nlp_ner(dataset["train"][0]["text"])

spacy.displacy.render(doc, style="ent", jupyter=True)

Making prediction for [competition](https://codalab.lisn.upsaclay.fr/competitions/18459). 

In [ ]:
import json

test_data = []

with open("test_in.jsonl", "r") as f:
    for line in f:
        test_data.append(json.loads(line))

len(test_data)

65

In [ ]:
results = []

for test in test_data:
    doc = nlp_ner(test["senences"])
    ners = list(map(lambda x: (x.start_char, x.end_char, x.label_), doc.ents))
    result = {
        "id": test["id"],
        "ners": ners
    }
    results.append(result)

len(results)

65

In [60]:
with open("test.jsonl", "w") as f:
    lines = list(map(json.dumps, results))
    f.write("\n".join(lines))